GitHub<br>
https://github.com/seasonSH/SemanticStyleGAN<br>
論文<br>
https://arxiv.org/abs/2112.02236<br>
<br>
<a href="https://colab.research.google.com/github/kaz12tech/ai_demos/blob/master/SemanticStyleGAN.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 環境セットアップ

## GPU確認

In [ ]:
!nvidia-smi

## Githubからソースコード取得

In [ ]:
%cd /content

!git clone https://github.com/seasonSH/SemanticStyleGAN.git
# for align face
!git clone https://github.com/adamian98/pulse.git

## ライブラリのインストール

In [ ]:
%cd /content/SemanticStyleGAN

# ninja
!wget https://github.com/ninja-build/ninja/releases/download/v1.8.2/ninja-linux.zip > /dev/null
!sudo unzip ninja-linux.zip -d /usr/local/bin/ > /dev/null
!sudo update-alternatives --install /usr/bin/ninja ninja /usr/local/bin/ninja 1 --force > /dev/null

!pip install -r requirements.txt

## ライブラリのインポート

In [ ]:
%cd /content/SemanticStyleGAN

import os
import argparse
import shutil
import numpy as np
import imageio
import time
import torch

from models import make_model
from visualize.utils import generate, cubic_spline_interpolate

from criteria.lpips import lpips

# 学習済みモデルのセットアップ

In [ ]:
%cd /content/SemanticStyleGAN
!mkdir pretrained

if not os.path.exists('./pretrained/CelebAMask-HQ-512x512.pt'):
  !wget -c https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/CelebAMask-HQ-512x512.pt \
        -O ./pretrained/CelebAMask-HQ-512x512.pt
if not os.path.exists('./pretrained/BitMoji-512x512.pt'):
  !wget -c https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/BitMoji-512x512.pt \
        -O ./pretrained/BitMoji-512x512.pt
if not os.path.exists('./pretrained/MetFaces-512x512.pt'):
  !wget -c https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/MetFaces-512x512.pt \
        -O ./pretrained/MetFaces-512x512.pt
if not os.path.exists('./pretrained/Toonify-512x512.pt'):
  !wget -c https://github.com/seasonSH/SemanticStyleGAN/releases/download/1.0.0/Toonify-512x512.pt \
        -O ./pretrained/Toonify-512x512.pt

# Random Synthesis

In [ ]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/CelebAMask-HQ-512x512.pt'
args.outdir = './results/samples'
args.batch = 8
args.sample = 20
args.truncation = 0.7
args.truncation_mean = 10000
args.save_latent = True
args.device = 'cuda'

if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating images ...")
start_time = time.time()
with torch.no_grad():
  styles = model.style(torch.randn(args.sample, model.style_dim, device=args.device))
  styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  images, segs = generate(model, styles, mean_latent=mean_latent, batch_size=args.batch)
  for i in range(len(images)):
    imageio.imwrite(f"{args.outdir}/{str(i).zfill(6)}_img.jpg", images[i])
    imageio.imwrite(f"{args.outdir}/{str(i).zfill(6)}_seg.jpg", segs[i])
    if args.save_latent:
      np.save(f'{args.outdir}/{str(i).zfill(6)}_latent.npy', styles[i:i+1].cpu().numpy())
print(f"Average speed: {(time.time() - start_time)/(args.sample)}s")

# Generate Video

In [ ]:
latent_dict_celeba = {
  2:  "bcg_1",
  3:  "bcg_2",
  4:  "face_shape",
  5:  "face_texture",
  6:  "eye_shape",
  7:  "eye_texture",
  8:  "eyebrow_shape",
  9:  "eyebrow_texture",
  10: "mouth_shape",
  11: "mouth_texture",
  12: "nose_shape",
  13: "nose_texture",
  14: "ear_shape",
  15: "ear_texture",
  16: "hair_shape",
  17: "hair_texture",
  18: "neck_shape",
  19: "neck_texture",
  20: "cloth_shape",
  21: "cloth_texture",
  22: "glass",
  24: "hat",
  26: "earing",
  0:  "coarse_1",
  1:  "coarse_2",
}

In [ ]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/CelebAMask-HQ-512x512.pt'
args.latent = './results/samples/000000_latent.npy'
args.outdir = './results/videos'
args.batch = 8
args.sample = 10
args.steps = 160
args.truncation = 0.7
args.truncation_mean = 10000
args.dataset_name = 'celeba'
args.device = 'cuda'


if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating original image ...")
with torch.no_grad():
  if args.latent is None:
    styles = model.style(torch.randn(1, model.style_dim, device=args.device))
    styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  else:
    styles = torch.tensor(np.load(args.latent), device=args.device)
  if styles.ndim == 2:
    assert styles.size(1) == model.style_dim
    styles = styles.unsqueeze(1).repeat(1, model.n_latent, 1)
  images, segs = generate(model, styles, mean_latent=mean_latent, randomize_noise=False)
  imageio.imwrite(f'{args.outdir}/image.jpeg', images[0])
  imageio.imwrite(f'{args.outdir}/seg.jpeg', segs[0])
print("Generating videos ...")
if args.dataset_name == "celeba":
  latent_dict = latent_dict_celeba
else:
  raise ValueError("Unknown dataset name: f{args.dataset_name}")

with torch.no_grad():
  for latent_index, latent_name in latent_dict.items():
    styles_new = styles.repeat(args.sample, 1, 1)
    mix_styles = model.style(torch.randn(args.sample, 512, device=args.device))
    mix_styles[-1] = mix_styles[0]
    mix_styles = args.truncation * mix_styles + (1-args.truncation) * mean_latent.unsqueeze(0)
    mix_styles = mix_styles.unsqueeze(1).repeat(1,model.n_latent,1)
    styles_new[:,latent_index] = mix_styles[:,latent_index]
    styles_new = cubic_spline_interpolate(styles_new, step=args.steps)
    images, segs = generate(model, styles_new, mean_latent=mean_latent, randomize_noise=False, batch_size=args.batch)
    frames = [np.concatenate((img,seg),1) for (img,seg) in zip(images,segs)]
    imageio.mimwrite(f'{args.outdir}/{latent_index:02d}_{latent_name}.mp4', frames, fps=20)
    print(f"{args.outdir}/{latent_index:02d}_{latent_name}.mp4")

# videoの表示

In [ ]:
!pip install moviepy

In [ ]:
%cd /content/SemanticStyleGAN

from moviepy.editor import *
from moviepy.video.fx.resize import resize

concat_video = './results/videos/concat.mp4'

clip1 = VideoFileClip('/content/SemanticStyleGAN/results/videos/04_face_shape.mp4')
clip2 = VideoFileClip('/content/SemanticStyleGAN/results/videos/06_eye_shape.mp4')
clip3 = VideoFileClip('/content/SemanticStyleGAN/results/videos/10_mouth_shape.mp4')
clip4 = VideoFileClip('/content/SemanticStyleGAN/results/videos/18_neck_shape.mp4')
clip5 = VideoFileClip('/content/SemanticStyleGAN/results/videos/16_hair_shape.mp4')
clip6 = VideoFileClip('/content/SemanticStyleGAN/results/videos/17_hair_texture.mp4')

concat = clips_array([[clip1, clip2], [clip3, clip4], [clip5, clip6]])
concat.write_videofile(concat_video)

In [ ]:
concat = resize(concat, width=640)
concat.ipython_display()

# invert

In [ ]:
%cd /content/SemanticStyleGAN
!rm -rf test_img
!mkdir -p test_img/src test_img/align

!wget -c https://www.pakutaso.com/shared/img/thumb/kuchikomi725_TP_V.jpg \
      -O ./test_img/src/test1.jpg

%cd /content/pulse
!python align_face.py \
  -input_dir /content/SemanticStyleGAN/test_img/src \
  -output_dir /content/SemanticStyleGAN/test_img/align \
  -output_size 512 \
  -seed 12 \

In [ ]:
%cd /content/SemanticStyleGAN
!cp visualize/invert.py invert.py

!python invert.py \
  --ckpt pretrained/CelebAMask-HQ-512x512.pt \
  --imgdir test_img/align \
  --outdir results/inversion \
  --size 512

# generate image Bitmoji

In [ ]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/BitMoji-512x512.pt'
args.latent = './results/inversion/latent/test1_0.npy'
args.outdir = './results/style_BitMoji'
args.truncation = 0.7
args.truncation_mean = 10000
args.device = 'cuda'

if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating original image ...")
with torch.no_grad():
  if args.latent is None:
    styles = model.style(torch.randn(1, model.style_dim, device=args.device))
    styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  else:
    styles = torch.tensor(np.load(args.latent), device=args.device)
  if styles.ndim == 2:
    assert styles.size(1) == model.style_dim
    styles = styles.unsqueeze(1).repeat(1, model.n_latent, 1)
  images, segs = generate(model, styles, mean_latent=mean_latent, randomize_noise=False)
  imageio.imwrite(f'{args.outdir}/image.jpeg', images[0])
  imageio.imwrite(f'{args.outdir}/seg.jpeg', segs[0])

# generate image MetfFaces

In [ ]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/MetFaces-512x512.pt'
args.latent = './results/inversion/latent/test1_0.npy'
args.outdir = './results/style_MetFaces'
args.truncation = 0.7
args.truncation_mean = 10000
args.device = 'cuda'

if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating original image ...")
with torch.no_grad():
  if args.latent is None:
    styles = model.style(torch.randn(1, model.style_dim, device=args.device))
    styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  else:
    styles = torch.tensor(np.load(args.latent), device=args.device)
  if styles.ndim == 2:
    assert styles.size(1) == model.style_dim
    styles = styles.unsqueeze(1).repeat(1, model.n_latent, 1)
  images, segs = generate(model, styles, mean_latent=mean_latent, randomize_noise=False)
  imageio.imwrite(f'{args.outdir}/image.jpeg', images[0])
  imageio.imwrite(f'{args.outdir}/seg.jpeg', segs[0])

In [ ]:
%cd /content/SemanticStyleGAN

args = argparse.ArgumentParser()

args.ckpt = './pretrained/Toonify-512x512.pt'
args.latent = './results/inversion/latent/test1_0.npy'
args.outdir = './results/style_Toonify'
args.truncation = 0.7
args.truncation_mean = 10000
args.device = 'cuda'

if os.path.exists(args.outdir):
  shutil.rmtree(args.outdir)
os.makedirs(args.outdir)

print("Loading model ...")
ckpt = torch.load(args.ckpt)
model = make_model(ckpt['args'])
model.to(args.device)
model.eval()
model.load_state_dict(ckpt['g_ema'])
mean_latent = model.style(torch.randn(args.truncation_mean, model.style_dim, device=args.device)).mean(0)

print("Generating original image ...")
with torch.no_grad():
  if args.latent is None:
    styles = model.style(torch.randn(1, model.style_dim, device=args.device))
    styles = args.truncation * styles + (1-args.truncation) * mean_latent.unsqueeze(0)
  else:
    styles = torch.tensor(np.load(args.latent), device=args.device)
  if styles.ndim == 2:
    assert styles.size(1) == model.style_dim
    styles = styles.unsqueeze(1).repeat(1, model.n_latent, 1)
  images, segs = generate(model, styles, mean_latent=mean_latent, randomize_noise=False)
  imageio.imwrite(f'{args.outdir}/image.jpeg', images[0])
  imageio.imwrite(f'{args.outdir}/seg.jpeg', segs[0])